In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [2]:
! rm -r sketch2ppt
! git clone https://github.com/fateh288/sketch2ppt.git



Cloning into 'sketch2ppt'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 80 (delta 36), reused 64 (delta 26), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import h5py
def load_data():
    f = h5py.File('sketch2ppt/hdf5/shapes_classification.hdf5', 'r')
    return (f['train_img'][...],f['train_labels'][...]),(f['test_img'][...],f['test_labels'][...])

In [5]:
(train_images,train_labels),(test_images,test_labels) = load_data()
#train_images, test_images = train_images / 255.0, test_images / 255.0
print(train_images.shape, type(train_images))
print(train_labels.shape, type(train_labels))

(200, 64, 64, 1) <class 'numpy.ndarray'>
(200,) <class 'numpy.ndarray'>


In [6]:
IMG_SIZE = 64

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.8),
])
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

model = models.Sequential()
model.add(resize_and_rescale)
#model.add(data_augmentation)
# model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [9]:
custom_adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=custom_adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=500,
                    validation_data=(test_images, test_labels))


Epoch 1/500
7/7 [==============================] - 0s 20ms/step - loss: 1.2036 - accuracy: 0.7250 - val_loss: 1.5440 - val_accuracy: 0.3800
Epoch 2/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0903 - accuracy: 0.9250 - val_loss: 1.5205 - val_accuracy: 0.3200
Epoch 3/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0727 - accuracy: 0.9250 - val_loss: 1.4944 - val_accuracy: 0.4200
Epoch 4/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0787 - accuracy: 0.9200 - val_loss: 1.5377 - val_accuracy: 0.3000
Epoch 5/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0670 - accuracy: 0.9400 - val_loss: 1.5440 - val_accuracy: 0.3200
Epoch 6/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0830 - accuracy: 0.9150 - val_loss: 1.5041 - val_accuracy: 0.3600
Epoch 7/500
7/7 [==============================] - 0s 5ms/step - loss: 1.0721 - accuracy: 0.9200 - val_loss: 1.4809 - val_accuracy: 0.4800
Epoch 8/500
7/7 [=========

In [11]:
model.save('/content/gdrive/My Drive/saved_model_sketch2ppt/model_fully_connected')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_sketch2ppt/model_fully_connected/assets


In [12]:
loaded_model = tf.keras.models.load_model('/content/gdrive/My Drive/saved_model_sketch2ppt/model_fully_connected')
loaded_model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 64, 64, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               524416    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                